In [33]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, f1_score, recall_score, precision_score, confusion_matrix, make_scorer
from sklearn.svm import SVC 
from sklearn.pipeline import make_pipeline

from scipy.stats import chi2_contingency
from pipelines import preprocessor, preprocessor2
from preprocess import extra_features, separate

In [2]:
train=pd.read_csv('Datasets/train.csv')
train=extra_features(train)
train.drop_duplicates(keep="last", inplace=True)

<IPython.core.display.Javascript object>

In [3]:
separate(train)

Target Column: ['cost_category']

Numerical Columns: ['night_mainland', 'night_zanzibar', 'total_nights']

Categorical Columns: ['age_group', 'travel_with', 'tour_arrangement', 'package_transport_int', 'package_accomodation', 'package_food', 'package_transport_tz', 'package_sightseeing', 'package_guided_tour', 'package_insurance', 'first_trip_tz', 'trip_length', 'group_size']

Text Columns: ['country', 'purpose', 'main_activity', 'info_source']


In [4]:
X=train.drop('cost_category', axis=1)
y=train['cost_category']

In [5]:
col_drop=[]

for col in X.columns:
    
    ct=pd.crosstab(y, X[col])
    chi2, P, _ , _ = chi2_contingency(ct)
    
    if P >= 0.05:
        col_drop.append(col)
    elif chi2 <= 1000:
            col_drop.append(col)
     
    #print(f"Column: {col}\nChi_Test: {chi2}\nP_Value: {P}\n---------------")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
col_drop

['trip_length', 'group_size']

In [7]:
X.drop(columns=col_drop, inplace=True)

In [8]:
X=preprocessor2(X)

In [9]:
X.shape

(18506, 187)

In [10]:
le=LabelEncoder()
y=le.fit_transform(y)

In [11]:
X_train,X_test,y_train,y_test=train_test_split(X, y, shuffle=True, stratify=y, random_state=1)

In [12]:
cl=SVC(gamma=10, C= 0.01, kernel='linear')
cl.fit(X_train, y_train)

SVC(C=0.01, gamma=10, kernel='linear')

In [13]:
y_pred=cl.predict(X_test)

In [14]:
print(classification_report(y_test, y_pred, zero_division=0, target_names=le.classes_))

              precision    recall  f1-score   support

   High Cost       0.44      0.39      0.41       920
 Higher Cost       0.62      0.77      0.69      1216
Highest Cost       0.00      0.00      0.00        90
    Low Cost       0.00      0.00      0.00       391
  Lower Cost       0.58      0.51      0.54       642
 Normal Cost       0.52      0.65      0.58      1368

    accuracy                           0.55      4627
   macro avg       0.36      0.39      0.37      4627
weighted avg       0.48      0.55      0.51      4627



In [15]:
folds= StratifiedKFold(n_splits=5, shuffle=True, random_state=42 )

scores = []

for fold, (train_idx, val_idx) in enumerate (folds.split(X,y)):
    X_train, X_test = X[train_idx], X[val_idx]
    y_train, y_test = y[train_idx], y[val_idx]
    
    cl.fit(X_train, y_train)
    score = cl.score(X_test, y_test)
    scores.append(score)
    print('Fold: %2d, Class Dist: %s, Accuracy: %.3f' % (fold+1, np.bincount(y_train), score))

<IPython.core.display.Javascript object>

Fold:  1, Class Dist: [2942 3892  287 1253 2054 4376], Accuracy: 0.544


<IPython.core.display.Javascript object>

Fold:  2, Class Dist: [2942 3892  288 1252 2054 4377], Accuracy: 0.544


<IPython.core.display.Javascript object>

Fold:  3, Class Dist: [2942 3892  287 1253 2054 4377], Accuracy: 0.540


<IPython.core.display.Javascript object>

Fold:  4, Class Dist: [2943 3892  287 1253 2053 4377], Accuracy: 0.521


<IPython.core.display.Javascript object>

Fold:  5, Class Dist: [2943 3892  287 1253 2053 4377], Accuracy: 0.546


In [16]:
print('\nCV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


CV accuracy: 0.539 +/- 0.009


In [17]:
cl.fit(X_train, y_train)

y_pred=cl.predict(X_test)

print(classification_report(y_test, y_pred, zero_division=True, target_names= le.classes_))

              precision    recall  f1-score   support

   High Cost       0.43      0.37      0.40       735
 Higher Cost       0.61      0.77      0.68       973
Highest Cost       1.00      0.00      0.00        72
    Low Cost       1.00      0.00      0.00       313
  Lower Cost       0.62      0.53      0.57       514
 Normal Cost       0.52      0.67      0.59      1094

    accuracy                           0.55      3701
   macro avg       0.70      0.39      0.37      3701
weighted avg       0.59      0.55      0.51      3701



In [18]:
scores=cross_val_score(cl, X_train, y_train, scoring='accuracy', cv=5)
scores.mean()

<IPython.core.display.Javascript object>

0.5371833839918947

In [34]:
%%time 
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

svm_pipe = make_pipeline(StandardScaler(),
                        SVC(random_state=123))

score = make_scorer(precision_score(average='weighted'))

values = [0.01, 0.1, 10.0]
param_grid = [{'svc__C': values,
              'svc__gamma':values,
               'svc__kernel':['rbf']},
               {'svc__C': values,
                'svc__gamma': values,
                'svc__kernel': ['linear']}]

gs = GridSearchCV(estimator=svm_pipe,
                  param_grid=param_grid,
                  scoring='accuracy', cv=3, verbose=100,
                  n_jobs=1)


gs.fit(X_train, y_train)

<IPython.core.display.Javascript object>

Fitting 3 folds for each of 18 candidates, totalling 54 fits
[CV 1/3; 1/18] START svc__C=0.01, svc__gamma=0.01, svc__kernel=rbf..............


C:\Users\User\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\User\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 234, in __call__
    return self._score(
  File "C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1954, in precision_score
    p, _, _, _ = precision_recall_fscore_support(
  File "C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1573, in precision_recall_fscore_support
    l

[CV 1/3; 1/18] END svc__C=0.01, svc__gamma=0.01, svc__kernel=rbf;, score=nan total time= 1.0min
[CV 2/3; 1/18] START svc__C=0.01, svc__gamma=0.01, svc__kernel=rbf..............


C:\Users\User\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\User\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 234, in __call__
    return self._score(
  File "C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1954, in precision_score
    p, _, _, _ = precision_recall_fscore_support(
  File "C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1573, in precision_recall_fscore_support
    l

...

C:\Users\User\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\User\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 234, in __call__
    return self._score(
  File "C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1954, in precision_score
    p, _, _, _ = precision_recall_fscore_support(
  File "C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1573, in precision_recall_fscore_support
    l

[CV 1/3; 16/18] END svc__C=10.0, svc__gamma=0.01, svc__kernel=linear;, score=nan total time=303.1min
[CV 2/3; 16/18] START svc__C=10.0, svc__gamma=0.01, svc__kernel=linear..........


C:\Users\User\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\User\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 234, in __call__
    return self._score(
  File "C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1954, in precision_score
    p, _, _, _ = precision_recall_fscore_support(
  File "C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1573, in precision_recall_fscore_support
    l

[CV 2/3; 16/18] END svc__C=10.0, svc__gamma=0.01, svc__kernel=linear;, score=nan total time=315.2min
[CV 3/3; 16/18] START svc__C=10.0, svc__gamma=0.01, svc__kernel=linear..........


KeyboardInterrupt: 

In [35]:
print(gs.best_score_)

AttributeError: 'GridSearchCV' object has no attribute 'best_score_'

In [31]:
print(gs.best_params_)

{'svc__C': 10.0, 'svc__gamma': 0.1, 'svc__kernel': 'rbf'}


In [32]:
print(gs.best_estimator_)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pca', PCA(n_components=4)),
                ('svc', SVC(C=10.0, gamma=0.1, random_state=123))])


In [ ]:

        """
        clf=SVC()

        C_values = [0.1, 1, 10, 100, 1000]
        for C in C_values:
            clf.C = C
            clf.fit(X_train, y_train)
            print("C = {}: accuracy = {}".format(C, clf.score(X_test, y_test)))

        # Tune the kernel parameter
        kernels = [ "linear", "rbf"]
        for kernel in kernels:
            clf.kernel = kernel
            clf.fit(X_train, y_train)
            print("Kernel = {}: accuracy = {}".format(kernel, clf.score(X_test, y_test)))

        # Tune the decision_function_shape parameter
        decision_function_shapes = ["ovo", "ovr"]
        for decision_function_shape in decision_function_shapes:
            clf.decision_function_shape = decision_function_shape
            clf.fit(X_train, y_train)
            print("Decision_function_shape = {}: accuracy = {}".format(decision_function_shape, clf.score(X_test, y_test)))

        # Tune the class_weight parameter
        class_weights = {0: 1, 1: 2}
        clf.class_weight = class_weights
        clf.fit(X_train, y_train)
        print("Class_weight = {}: accuracy = {}".format(class_weights, clf.score(X_test, y_test)))
        """
